# PGM Calculation Engine Examples

These examples show how to interact with the [power-grid-model](https://github.com/PowerGridModel/power-grid-model) calculation engine to perform load flow calculations. For a detailed documentation on the calculation engine please refer to [power-grid-model docs](https://power-grid-model.readthedocs.io/en/stable/).


## Single Load Flow example

We will demonstrate how to use the PGMCoreInterface to perform Power Flow calculations on a `Grid` object.
In the examples the `RadialGridGenerator` is used to create randomised input networks.


In [1]:
from power_grid_model_ds import Grid, PowerGridModelInterface
from power_grid_model_ds.generators import RadialGridGenerator

In [2]:
grid_generator = RadialGridGenerator(grid_class=Grid, nr_nodes=5, nr_sources=1, nr_nops=0)
grid = grid_generator.run(seed=0)

print("Created Grid:")
print(grid.branches)

core_interface = PowerGridModelInterface(grid=grid)

# Create input from grid and calculate power flow
core_interface.create_input_from_grid()
output = core_interface.calculate_power_flow()

print("Power Flow Results:")
display(output["node"])
display(output["line"])

Created Grid:
 id | from_node | to_node | from_status | to_status | feeder_branch_id | feeder_node_id | is_feeder 
 18 |     16    |    5    |      1      |     1     |        0         |       0        |   False   
 19 |     5     |    3    |      1      |     1     |        0         |       0        |   False   
 20 |     5     |    4    |      1      |     1     |        0         |       0        |   False   
 21 |     4     |    2    |      1      |     1     |        0         |       0        |   False   
 22 |     16    |    1    |      1      |     1     |        0         |       0        |   False   
Power Flow Results:


array([( 1, 1, 1.00022909, 10502.40547296, 1.99595527e-04,   318145.99999988,  30629.        ),
       ( 2, 1, 1.00181391, 10519.04604187, 8.18373270e-05,    85535.99999995,  24049.        ),
       ( 3, 1, 1.00120197, 10512.62069113, 3.07703301e-04,   913761.00000028,  16071.00000004),
       ( 4, 1, 1.00164458, 10517.26805716, 1.15203146e-04,   176577.0000001 ,  -9407.        ),
       ( 5, 1, 1.00008458, 10500.88803896, 2.02276426e-04,   536068.99999626, -31487.99999998),
       (16, 1, 1.00002312, 10500.2427774 , 2.01540352e-04, -2028475.46388834, -29731.71601113)],
      dtype={'names': ['id', 'energized', 'u_pu', 'u', 'u_angle', 'p', 'q'], 'formats': ['<i4', 'i1', '<f8', '<f8', '<f8', '<f8', '<f8'], 'offsets': [0, 4, 8, 16, 24, 32, 40], 'itemsize': 48, 'aligned': True})

array([(18, 1, 0.30944115, -1710395.03728345,    891.59535891, 94.04517498, 1710395.26966902, 1710500.14523965,  -890.39109601, 94.04517498, 1710500.37698364),
       (19, 1, 0.11782317,  -912742.88075081, -15956.83635493, 50.19126661,  912882.35112075,  913761.00000028, 16071.00000004, 50.19126661,  913902.31543777),
       (20, 1, 0.0109873 ,  -261688.26448931, -14640.77254911, 14.41040626,  262097.50092716,  262097.73992698, 14640.78859395, 14.41040626,  262506.33890534),
       (21, 1, 0.04259208,   -85520.73992694, -24047.78859394,  4.87677287,   88837.45321603,   85535.99999995, 24049.        ,  4.87677287,   88852.47153001),
       (22, 1, 0.02406958,  -318080.42660057, -30623.31137002, 17.57035082,  319551.16176548,  318145.99999988, 30629.        , 17.57035082,  319616.97851792)],
      dtype={'names': ['id', 'energized', 'loading', 'p_from', 'q_from', 'i_from', 's_from', 'p_to', 'q_to', 'i_to', 's_to'], 'formats': ['<i4', 'i1', '<f8', '<f8', '<f8', '<f8', '<f8', '<f8', '<f8',

## Updating the Grid object

If you want to store specific outputs of the calculation in the `Grid` object you can specify the columns in extended arrays. For a more detailed how to on extending arrays there is a seperate example.


In [3]:
from dataclasses import dataclass

import numpy as np
from numpy.typing import NDArray

from power_grid_model_ds import GraphContainer, Grid
from power_grid_model_ds.arrays import LineArray, NodeArray


class ExtendedNodeArray(NodeArray):
    """Extends the node array with an output value"""

    _defaults = {"u": 0}

    u: NDArray[np.float64]


class ExtendedLineArray(LineArray):
    """Extends the line array with an output value"""

    _defaults = {"i_from": 0}

    i_from: NDArray[np.float64]


@dataclass
class ExtendedGrid(Grid):
    """
    This is my own grid to extend.
    """

    node: ExtendedNodeArray
    line: ExtendedLineArray
    graphs: GraphContainer

In [4]:
grid_generator = RadialGridGenerator(grid_class=ExtendedGrid, nr_nodes=5, nr_sources=1, nr_nops=0)
grid = grid_generator.run(seed=0)

core_interface = PowerGridModelInterface(grid=grid)
core_interface.create_input_from_grid()
core_interface.calculate_power_flow()
core_interface.update_grid()

print(grid.node)
print(grid.line)

 id | u_rated | node_type | feeder_branch_id | feeder_node_id |     u     
 1  | 10500.0 |     0     |        0         |       0        |10502.405..
 2  | 10500.0 |     0     |        0         |       0        |10519.046..
 3  | 10500.0 |     0     |        0         |       0        |10512.620..
 4  | 10500.0 |     0     |        0         |       0        |10517.268..
 5  | 10500.0 |     0     |        0         |       0        |10500.888..
 16 | 10500.0 |     1     |   -2147483648    |  -2147483648   |10500.242..
 id | from_node | to_node | from_status | to_status | feeder_branch_id | feeder_node_id | is_feeder |   r1  |   x1  |  c1 | tan1 |   i_n    | i_from 
 18 |     16    |    5    |      1      |     1     |        0         |       0        |   False   |0.003..|4.538..| 0.0 | 0.0  |303.9194..|94.045..
 19 |     5     |    3    |      1      |     1     |        0         |       0        |   False   |0.134..|0.015..| 0.0 | 0.0  |425.9880..|50.191..
 20 |     5     |    4   

## Batch Load Flow Example

To use the `PGM` batch calculation functionality you can supply a dictionary with update data to the `calculate_power_flow` function.


In [5]:
from power_grid_model import initialize_array

grid_generator = RadialGridGenerator(grid_class=Grid, nr_nodes=5, nr_sources=1, nr_nops=0)
grid = grid_generator.run(seed=0)
core_interface = PowerGridModelInterface(grid=grid)

update_sym_load = initialize_array("update", "sym_load", (10, len(grid.sym_load)))
update_sym_load["id"] = [grid.sym_load.id.tolist()]
update_sym_load["p_specified"] = [grid.sym_load.p_specified.tolist()] * np.linspace(0, 1, 10).reshape(-1, 1)
update_sym_load["q_specified"] = [grid.sym_load.q_specified.tolist()] * np.linspace(0, 1, 10).reshape(-1, 1)
update_data = {
    "sym_load": update_sym_load,
}
output = core_interface.calculate_power_flow(update_data=update_data)

# Results have been calculated for all 10 scenarios
display(output["line"])

array([[(18, 1, 0.00000000e+00,  0.00000000e+00, -0.00000000e+00, 0.00000000e+00, 0.00000000e+00,  0.00000000e+00, -0.00000000e+00, 0.00000000e+00, 0.00000000e+00),
        (19, 1, 2.93489658e-14, -2.27373675e-07,  1.20159855e-23, 1.25023098e-11, 2.27373675e-07,  2.27373675e-07, -9.32440637e-24, 1.25023098e-11, 2.27373675e-07),
        (20, 1, 9.53298261e-15,  2.27373675e-07, -2.37049963e-09, 1.25029892e-11, 2.27386032e-07, -2.27373675e-07,  2.37049963e-09, 1.25029892e-11, 2.27386032e-07),
        (21, 1, 5.47019878e-14,  1.13686838e-07, -7.10542736e-09, 6.26335228e-12, 1.13908666e-07, -1.13686838e-07,  7.10542736e-09, 6.26335228e-12, 1.13908666e-07),
        (22, 1, 3.43205974e-14, -4.54747351e-07, -2.84217094e-08, 2.50534091e-11, 4.55634664e-07,  4.54747351e-07,  2.84217094e-08, 2.50534091e-11, 4.55634664e-07)],
       [(18, 1, 3.44107076e-02, -1.90196733e+05,  8.75536458e+01, 1.04580823e+01, 1.90196753e+05,  1.90198033e+05, -8.75387538e+01, 1.04580823e+01, 1.90198053e+05),
        (

## Create grid from input data example


In [7]:
from power_grid_model_ds.arrays import SourceArray, SymLoadArray, TransformerArray

input_data = core_interface.create_input_from_grid()

array_mapping = {
    "node": NodeArray,
    "line": LineArray,
    "sym_load": SymLoadArray,
    "source": SourceArray,
    "transformer": TransformerArray,
}

core_interface = PowerGridModelInterface(input_data=input_data)
output = core_interface.create_grid_from_input_data(
    array_mapping=array_mapping,
)
print(input_data["node"])
print(output.node)

[( 1, 10500.) ( 2, 10500.) ( 3, 10500.) ( 4, 10500.) ( 5, 10500.)
 (16, 10500.)]
 id | u_rated | node_type | feeder_branch_id | feeder_node_id 
 1  | 10500.0 |     0     |   -2147483648    |  -2147483648   
 2  | 10500.0 |     0     |   -2147483648    |  -2147483648   
 3  | 10500.0 |     0     |   -2147483648    |  -2147483648   
 4  | 10500.0 |     0     |   -2147483648    |  -2147483648   
 5  | 10500.0 |     0     |   -2147483648    |  -2147483648   
 16 | 10500.0 |     0     |   -2147483648    |  -2147483648   
